# 標記型態

## 種類
* 正常 cdef, pyx 檔
* py/pxd 檔
* pure python mode
* python3 type hints

In [1]:
%load_ext Cython

_1.1_ 沒有標記的程式碼

In [16]:
%%cython -a
def f(x):
    z = 3
    y = x
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

_1.2_ 加上輸入型態， 可以看到 `i,z,y` 都還是 python variables

In [23]:
%%cython -a
def f(int x):
    z = 3
    y = x
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

_1.3_ 將 i, y, z 加上型態

In [28]:
%%cython -a
def f(int x):
    cdef int i, y, z
    z = 3
    y = x
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

_1.4_ Type Inference

In [59]:
%%cython -a
# cython: infer_types=True
def f(int x):    
    y, z = x, 3    
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

_1.5_ Play with type inference

In [75]:
%%cython -a -f
# cython: infer_types=True
# cython: infer_types.verbose=True 
from math import sin
def f(int x):    
    z = 3
    y = x
    u = 5.3
    z += 1
    t = u + 3
    for i in range(10):
        z += 1

note: /home/tjw/.cache/ipython/cython/_cython_magic_9a79458e13875f6b2960f3e3c712135b.pyx:9:6: inferred 't' to be of type 'double'
note: /home/tjw/.cache/ipython/cython/_cython_magic_9a79458e13875f6b2960f3e3c712135b.pyx:6:6: inferred 'y' to be of type 'int'
note: /home/tjw/.cache/ipython/cython/_cython_magic_9a79458e13875f6b2960f3e3c712135b.pyx:5:6: inferred 'z' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_9a79458e13875f6b2960f3e3c712135b.pyx:10:10: inferred 'i' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_9a79458e13875f6b2960f3e3c712135b.pyx:7:6: inferred 'u' to be of type 'double'


_1.6_

_2.1_ Pure Python mode

In [111]:
%%cython -a -f
# cython: infer_types=True, infer_types.verbose=True 
import cython
@cython.locals(x='int')
def f(x):    
    y, z = x, 3    
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

note: /home/tjw/.cache/ipython/cython/_cython_magic_211224a90befd99052671773218b8155.pyx:7:5: inferred 'y' to be of type 'int'
note: /home/tjw/.cache/ipython/cython/_cython_magic_211224a90befd99052671773218b8155.pyx:7:9: inferred 'z' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_211224a90befd99052671773218b8155.pyx:8:10: inferred 'i' to be of type 'long'


_2.2_ 一覽表
* `@cython.cclass` == `cdef class`.
* `@cython.cfunc` == `cdef` function.
* `@cython.ccall` == `cpdef` function
* `v = cython.declare(type)` == `cdef type v`  for global variable
* `@cython.inline` == C inline 
* `@cython.return(type)` == function return type
* `T = cython.typedef(cython.p_int)`  == `ctypedef`
* `cython.compiled` 用來判斷是否用 cython 編譯

參考 http://docs.cython.org/src/tutorial/pure.html

_2.3_ Type Hinting

In [122]:
%%cython -a -f
# cython: infer_types=True, annotation_typing=True, infer_types.verbose=True 
def f(x: int):    
    y, z = x, 3    
    for i in range(5):
        z = z  + i * x
        y = y + x 
        print(i+z)

note: /home/tjw/.cache/ipython/cython/_cython_magic_184087ea539d798860a4a1d7972bb94e.pyx:3:9: inferred 'z' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_184087ea539d798860a4a1d7972bb94e.pyx:3:5: inferred 'y' to be of type 'int'
note: /home/tjw/.cache/ipython/cython/_cython_magic_184087ea539d798860a4a1d7972bb94e.pyx:4:10: inferred 'i' to be of type 'long'


_3.1_ .py/.pxd 和 .pyx

差別：
* .py 中的 def 可以被 pxd 中的 cdef 蓋過
* .py 有更多東西。有時會影響速度 參考 [worksheet](/notebooks/Worksheet-brainfuck.ipynb)  bf4/bfx4

_3.2_ 標記的順序

* 跟 visibility 有關
* 通常 cython 比較容易看到 python 的東西

In [ ]:
%%cython
# 用 public 和 cpdef 讓 python 看到 cython 的東西
cdef class AA:
    cdef public:
        int kkk
    cpdef xx():
        print("x")

_4._ Fused type (template)

In [154]:
%%cython -a
cimport cython
ctypedef fused mtype:
    int
    float
    str
cdef mtype x2(mtype var):
    return var*2

cdef int a = x2(1)
cdef float b = x2(2.0)
print(x2("aaa"))

In [21]:
%%cython -a
# cython: infer_types=True, boundscheck=False
# cython: infer_types.verbose=True 
cimport cython
import numpy
ctypedef fused mtype:
    int
    float
    double
    str
cdef mtype p(mtype[] var):
    s = var[0]
    i = 0
    while var[i]:
        s += var[i]
        i += 1
    return s
cdef int[4] a = [5,4,3,0]
print p(a)
cdef float[4] b = [5,4,3,0]
print p(b)

17
17.0


note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:11:6: inferred 's' to be of type 'int'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:12:6: inferred 'i' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:11:6: inferred 's' to be of type 'double'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:12:6: inferred 'i' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:12:6: inferred 'i' to be of type 'long'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:11:6: inferred 's' to be of type 'double'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx:11:6: inferred 's' to be of type 'str object'
note: /home/tjw/.cache/ipython/cython/_cython_magic_7293a1ce0a9efb1865c4d59bf05e6aac.pyx

_5._ Jedi Typer

In [ ]:
%loadpy examples/jedi_test.py

In [34]:
%%bash
cd examples
# pip install jedi
python jedi-typer.py jedi_test.py

Strange access on <Array of Node(atom, [<Operator: [>, <Number: 0>, <Operator: ]>])>: obj.
Don't touch this: obj of <eClass of <Class: A@25-27>> !


輸出的結果

In [ ]:
%loadpy examples/jedi_test.py_typed.py